# Week 4: Python Aggregations & Summary Statistics - Part 2
## Advanced GroupBy and Pivot Tables

**Wednesday Python Class - September 3, 2025**  
**Business Context**: Multi-dimensional Sales Analysis  
**Excel Concepts**: Pivot Tables, Advanced SUMIFS with multiple criteria

---

### Learning Objectives:
1. Master pandas GroupBy operations with multiple columns
2. Use advanced aggregation functions and custom functions
3. Create pivot tables for multi-dimensional analysis
4. Apply filtering with multiple conditions (Excel SUMIFS equivalent)
5. Build comprehensive business reports

### Today's Business Challenge:
Create a comprehensive sales performance dashboard that analyzes multiple dimensions simultaneously - exactly like what data analysts at major e-commerce companies do for executive reporting.

## Section 1: Setup and Data Preparation

In [ ]:
# Import libraries and load data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
df = pd.read_csv('../datasets/olist_sample_data.csv')
df['order_date'] = pd.to_datetime(df['order_date'])

# Add additional business columns for analysis
df['year'] = df['order_date'].dt.year
df['month'] = df['order_date'].dt.month
df['month_name'] = df['order_date'].dt.strftime('%B')
df['quarter'] = df['order_date'].dt.quarter
df['day_of_week'] = df['order_date'].dt.day_name()

# Create business segments
df['price_segment'] = pd.cut(df['price'], 
                            bins=[0, 100, 200, 300, float('inf')], 
                            labels=['Budget', 'Standard', 'Premium', 'Luxury'])

# Customer satisfaction categories
df['satisfaction_level'] = df['review_score'].map({
    1: 'Very Poor', 2: 'Poor', 3: 'Average', 4: 'Good', 5: 'Excellent'
})

print(f"Dataset enhanced with business dimensions!")
print(f"Shape: {df.shape}")
print(f"New columns: {['year', 'month', 'month_name', 'quarter', 'day_of_week', 'price_segment', 'satisfaction_level']}")

## Section 2: Advanced GroupBy Operations

### 2.1 Single Column GroupBy with Multiple Aggregations

In [ ]:
# Business Question: Complete performance analysis by customer state
# Excel equivalent: Pivot table with State as rows, multiple value calculations

print("=== COMPREHENSIVE STATE PERFORMANCE ANALYSIS ===")

# Method 1: Using agg() with dictionary
state_analysis = df.groupby('customer_state').agg({
    'price': ['count', 'sum', 'mean', 'median', 'std'],
    'freight_value': ['mean', 'sum'],
    'review_score': ['mean', 'count']
}).round(2)

# Flatten column names for easier reading
state_analysis.columns = ['Order_Count', 'Total_Revenue', 'Avg_Price', 'Median_Price', 'Price_Std',
                         'Avg_Freight', 'Total_Freight', 'Avg_Rating', 'Rated_Orders']

# Add calculated columns
state_analysis['Revenue_per_Order'] = state_analysis['Total_Revenue'] / state_analysis['Order_Count']
state_analysis['Freight_Rate_%'] = (state_analysis['Total_Freight'] / state_analysis['Total_Revenue'] * 100).round(2)

# Sort by total revenue
state_analysis = state_analysis.sort_values('Total_Revenue', ascending=False)

print(state_analysis.head(10))

# Business insights
print(f"\n=== KEY INSIGHTS ===")
top_state = state_analysis.index[0]
most_consistent = state_analysis['Price_Std'].idxmin()
best_rating = state_analysis['Avg_Rating'].idxmax()
lowest_freight = state_analysis['Freight_Rate_%'].idxmin()

print(f"🏆 Top Revenue State: {top_state} (R$ {state_analysis.loc[top_state, 'Total_Revenue']:,.2f})")
print(f"📊 Most Consistent Pricing: {most_consistent} (Std: R$ {state_analysis.loc[most_consistent, 'Price_Std']:.2f})")
print(f"⭐ Best Customer Satisfaction: {best_rating} ({state_analysis.loc[best_rating, 'Avg_Rating']:.2f}/5.0)")
print(f"🚚 Lowest Shipping Cost: {lowest_freight} ({state_analysis.loc[lowest_freight, 'Freight_Rate_%']:.2f}%)")

### 2.2 Multi-Column GroupBy Analysis

In [ ]:
# Business Question: How does performance vary by state AND price segment?
# Excel equivalent: Pivot table with State and Price Segment as row fields

print("=== STATE x PRICE SEGMENT ANALYSIS ===")

state_segment_analysis = df.groupby(['customer_state', 'price_segment']).agg({
    'price': ['count', 'sum', 'mean'],
    'review_score': 'mean'
}).round(2)

# Flatten column names
state_segment_analysis.columns = ['Orders', 'Revenue', 'Avg_Price', 'Avg_Rating']

# Add percentage of total orders
total_orders = df.shape[0]
state_segment_analysis['Order_Share_%'] = (state_segment_analysis['Orders'] / total_orders * 100).round(2)

# Show top combinations
top_combinations = state_segment_analysis.sort_values('Revenue', ascending=False).head(10)
print(top_combinations)

print(f"\n=== SEGMENT INSIGHTS ===")
# Best performing state-segment combination
best_combo = top_combinations.index[0]
print(f"Best State-Segment: {best_combo[0]} - {best_combo[1]}")
print(f"  Revenue: R$ {top_combinations.iloc[0]['Revenue']:,.2f}")
print(f"  Orders: {top_combinations.iloc[0]['Orders']:,} ({top_combinations.iloc[0]['Order_Share_%']}% of total)")

# Premium segment analysis
premium_segments = state_segment_analysis.loc[state_segment_analysis.index.get_level_values(1) == 'Premium']
if not premium_segments.empty:
    best_premium_state = premium_segments['Revenue'].idxmax()[0]
    print(f"\nBest Premium Market: {best_premium_state}")
    print(f"  Premium Revenue: R$ {premium_segments.loc[(best_premium_state, 'Premium'), 'Revenue']:,.2f}")

### 2.3 Time-Based Analysis with Multiple Dimensions

In [ ]:
# Business Question: How do sales trends vary by month and customer state?
# Excel equivalent: Complex pivot table with months and states

print("=== MONTHLY PERFORMANCE BY TOP STATES ===")

# Focus on top 3 states for clarity
top_3_states = df['customer_state'].value_counts().head(3).index.tolist()
df_top_states = df[df['customer_state'].isin(top_3_states)]

monthly_state_analysis = df_top_states.groupby(['month_name', 'customer_state']).agg({
    'price': ['count', 'sum', 'mean'],
    'review_score': 'mean'
}).round(2)

# Flatten columns
monthly_state_analysis.columns = ['Orders', 'Revenue', 'Avg_Price', 'Avg_Rating']

# Reset index for easier manipulation
monthly_state_df = monthly_state_analysis.reset_index()

# Show sample results
print(monthly_state_df.head(10))

# Create month-wise summary
print(f"\n=== MONTHLY TRENDS SUMMARY ===")
monthly_summary = df.groupby('month_name').agg({
    'price': ['count', 'sum'],
    'review_score': 'mean'
}).round(2)

monthly_summary.columns = ['Total_Orders', 'Total_Revenue', 'Avg_Rating']

# Sort by month order (not alphabetical)
month_order = ['January', 'February', 'March', 'April', 'May', 'June',
               'July', 'August', 'September', 'October', 'November', 'December']
monthly_summary = monthly_summary.reindex([m for m in month_order if m in monthly_summary.index])

print(monthly_summary)

# Find best and worst performing months
best_month = monthly_summary['Total_Revenue'].idxmax()
worst_month = monthly_summary['Total_Revenue'].idxmin()

print(f"\nBest Month: {best_month} (R$ {monthly_summary.loc[best_month, 'Total_Revenue']:,.2f})")
print(f"Worst Month: {worst_month} (R$ {monthly_summary.loc[worst_month, 'Total_Revenue']:,.2f})")

## Section 3: Custom Aggregation Functions

In [ ]:
# Business Question: Create custom business metrics for each category
# Excel equivalent: Complex calculated fields in pivot tables

def business_metrics(series):
    """Custom function to calculate business KPIs"""
    return pd.Series({
        'total_orders': len(series),
        'total_revenue': series.sum(),
        'avg_order_value': series.mean(),
        'revenue_concentration': series.std() / series.mean() if series.mean() > 0 else 0,  # Coefficient of variation
        'top_10_percent_revenue': series.nlargest(int(len(series) * 0.1)).sum() if len(series) >= 10 else series.sum(),
        'min_order': series.min(),
        'max_order': series.max(),
        'revenue_range': series.max() - series.min()
    })

print("=== CUSTOM BUSINESS METRICS BY CATEGORY ===")

category_metrics = df.groupby('category')['price'].apply(business_metrics).round(2)

# Sort by total revenue
category_metrics = category_metrics.sort_values('total_revenue', ascending=False)

print(category_metrics.head(10))

# Business insights from custom metrics
print(f"\n=== ADVANCED BUSINESS INSIGHTS ===")

# Most consistent category (lowest revenue concentration)
most_consistent = category_metrics['revenue_concentration'].idxmin()
most_volatile = category_metrics['revenue_concentration'].idxmax()
widest_range = category_metrics['revenue_range'].idxmax()

print(f"📊 Most Consistent Category: {most_consistent}")
print(f"   Coefficient of Variation: {category_metrics.loc[most_consistent, 'revenue_concentration']:.2f}")

print(f"\n📈 Most Volatile Category: {most_volatile}")
print(f"   Coefficient of Variation: {category_metrics.loc[most_volatile, 'revenue_concentration']:.2f}")

print(f"\n🎯 Widest Price Range: {widest_range}")
print(f"   Range: R$ {category_metrics.loc[widest_range, 'min_order']:.2f} - R$ {category_metrics.loc[widest_range, 'max_order']:.2f}")

## Section 4: Pivot Tables - Excel's Best Friend in Pandas

### 4.1 Basic Pivot Table

In [ ]:
# Business Question: Create a pivot table showing revenue by state and price segment
# Excel equivalent: Pivot table with States as rows, Price Segments as columns, Revenue as values

print("=== REVENUE PIVOT TABLE: STATES vs PRICE SEGMENTS ===")

revenue_pivot = pd.pivot_table(df, 
                              values='price', 
                              index='customer_state', 
                              columns='price_segment', 
                              aggfunc='sum', 
                              fill_value=0,
                              margins=True,
                              margins_name='TOTAL').round(2)

print(revenue_pivot)

# Calculate percentage distribution
print(f"\n=== PERCENTAGE DISTRIBUTION ===")
revenue_pivot_pct = pd.pivot_table(df, 
                                  values='price', 
                                  index='customer_state', 
                                  columns='price_segment', 
                                  aggfunc='sum', 
                                  fill_value=0,
                                  normalize=True).round(4) * 100

print(revenue_pivot_pct)

# Business insights from pivot table
print(f"\n=== PIVOT TABLE INSIGHTS ===")

# Find top performers in each segment
for segment in ['Budget', 'Standard', 'Premium', 'Luxury']:
    if segment in revenue_pivot.columns:
        top_state = revenue_pivot[segment].idxmax()
        if top_state != 'TOTAL':  # Exclude the totals row
            print(f"Top {segment} Market: {top_state} (R$ {revenue_pivot.loc[top_state, segment]:,.2f})")

### 4.2 Multi-Index Pivot Table

In [ ]:
# Business Question: Analyze orders and revenue by state, category, and satisfaction level
# Excel equivalent: Complex pivot table with multiple row and column fields

print("=== MULTI-DIMENSIONAL PIVOT ANALYSIS ===")

# Focus on top 3 states and top 5 categories for readability
top_states = df['customer_state'].value_counts().head(3).index
top_categories = df['category'].value_counts().head(5).index

df_focused = df[(df['customer_state'].isin(top_states)) & 
                (df['category'].isin(top_categories)) &
                (df['satisfaction_level'].notna())]

# Create multi-level pivot table
multi_pivot = pd.pivot_table(df_focused,
                            values=['price', 'review_score'],
                            index=['customer_state', 'category'],
                            columns='satisfaction_level',
                            aggfunc={'price': ['count', 'sum'], 'review_score': 'mean'},
                            fill_value=0).round(2)

print("Sample of multi-dimensional pivot table:")
print(multi_pivot.head(10))

# Simplified version for business reporting
print(f"\n=== SIMPLIFIED BUSINESS REPORT ===")
business_pivot = pd.pivot_table(df_focused,
                               values='price',
                               index=['customer_state', 'category'],
                               columns='satisfaction_level',
                               aggfunc='count',
                               fill_value=0,
                               margins=True)

print(business_pivot)

### 4.3 Advanced Pivot Table with Multiple Aggregations

In [ ]:
# Business Question: Executive dashboard with multiple KPIs in one view
# Excel equivalent: Pivot table with multiple value fields

print("=== EXECUTIVE DASHBOARD PIVOT TABLE ===")

# Create comprehensive dashboard pivot
dashboard_pivot = pd.pivot_table(df,
                                values=['price', 'freight_value', 'review_score'],
                                index='customer_state',
                                aggfunc={
                                    'price': ['count', 'sum', 'mean'],
                                    'freight_value': ['mean', 'sum'],
                                    'review_score': 'mean'
                                },
                                fill_value=0).round(2)

# Flatten column names for easier reading
dashboard_pivot.columns = ['Freight_Mean', 'Freight_Sum', 'Orders', 'Revenue', 'AOV', 'Rating']

# Add calculated metrics
dashboard_pivot['Freight_Rate_%'] = (dashboard_pivot['Freight_Sum'] / dashboard_pivot['Revenue'] * 100).round(2)
dashboard_pivot['Revenue_Share_%'] = (dashboard_pivot['Revenue'] / dashboard_pivot['Revenue'].sum() * 100).round(2)

# Sort by revenue and show top 10
dashboard_pivot = dashboard_pivot.sort_values('Revenue', ascending=False)
print(dashboard_pivot.head(10))

# Executive summary from the dashboard
print(f"\n=== EXECUTIVE SUMMARY ===")
total_revenue = dashboard_pivot['Revenue'].sum()
total_orders = dashboard_pivot['Orders'].sum()
avg_rating = (df['review_score'] * df.groupby('customer_state').size()).sum() / df['review_score'].notna().sum()

print(f"📊 OVERALL PERFORMANCE:")
print(f"   Total Revenue: R$ {total_revenue:,.2f}")
print(f"   Total Orders: {total_orders:,}")
print(f"   Overall Average Rating: {avg_rating:.2f}/5.0")

print(f"\n🏆 TOP PERFORMERS:")
top_3 = dashboard_pivot.head(3)
for i, (state, row) in enumerate(top_3.iterrows(), 1):
    print(f"   #{i} {state}: R$ {row['Revenue']:,.2f} ({row['Revenue_Share_%']}% of total)")

print(f"\n📈 EFFICIENCY METRICS:")
best_aov_state = dashboard_pivot['AOV'].idxmax()
lowest_freight_state = dashboard_pivot['Freight_Rate_%'].idxmin()
print(f"   Highest AOV: {best_aov_state} (R$ {dashboard_pivot.loc[best_aov_state, 'AOV']:.2f})")
print(f"   Lowest Freight Rate: {lowest_freight_state} ({dashboard_pivot.loc[lowest_freight_state, 'Freight_Rate_%']:.2f}%)")

## Section 5: Advanced Filtering and Conditional Analysis

In [ ]:
# Business Question: Analyze high-value customers with excellent satisfaction
# Excel equivalent: Multiple SUMIFS with complex criteria

print("=== HIGH-VALUE CUSTOMER ANALYSIS ===")

# Define high-value criteria
high_value_criteria = (
    (df['price'] >= 200) &  # Premium/Luxury orders only
    (df['review_score'] >= 4) &  # Good/Excellent satisfaction
    (df['customer_state'].isin(['SP', 'RJ', 'MG']))  # Major markets only
)

high_value_customers = df[high_value_criteria]

print(f"High-value customer segment: {len(high_value_customers):,} orders")
print(f"Percentage of total: {len(high_value_customers)/len(df)*100:.1f}%")
print(f"Revenue contribution: R$ {high_value_customers['price'].sum():,.2f}")
print(f"Revenue percentage: {high_value_customers['price'].sum()/df['price'].sum()*100:.1f}%")

# Analyze this segment by state and category
hv_analysis = high_value_customers.groupby(['customer_state', 'category']).agg({
    'price': ['count', 'sum', 'mean'],
    'review_score': 'mean'
}).round(2)

hv_analysis.columns = ['Orders', 'Revenue', 'AOV', 'Rating']
hv_analysis = hv_analysis.sort_values('Revenue', ascending=False)

print(f"\n=== HIGH-VALUE SEGMENT BREAKDOWN ===")
print(hv_analysis.head(10))

# Business recommendations
print(f"\n=== STRATEGIC RECOMMENDATIONS ===")
top_hv_combo = hv_analysis.index[0]
print(f"🎯 Focus Area: {top_hv_combo[0]} state - {top_hv_combo[1]} category")
print(f"   This combination generates R$ {hv_analysis.iloc[0]['Revenue']:,.2f} in high-value revenue")
print(f"   Average order value: R$ {hv_analysis.iloc[0]['AOV']:,.2f}")
print(f"   Customer satisfaction: {hv_analysis.iloc[0]['Rating']:.1f}/5.0")

## Section 6: Business Intelligence Report Generation

In [ ]:
# Business Question: Generate a comprehensive BI report for stakeholders
# This combines all techniques learned

def generate_business_report(df):
    """Generate comprehensive business intelligence report"""
    
    report = {}
    
    # Executive Summary
    report['executive_summary'] = {
        'total_orders': len(df),
        'total_revenue': df['price'].sum(),
        'avg_order_value': df['price'].mean(),
        'customer_satisfaction': df['review_score'].mean(),
        'satisfaction_rate': (df['review_score'] >= 4).sum() / df['review_score'].notna().sum() * 100
    }
    
    # Market Analysis
    market_analysis = df.groupby('customer_state').agg({
        'price': ['count', 'sum', 'mean'],
        'review_score': 'mean'
    }).round(2)
    market_analysis.columns = ['Orders', 'Revenue', 'AOV', 'Rating']
    market_analysis['Market_Share_%'] = (market_analysis['Revenue'] / market_analysis['Revenue'].sum() * 100).round(1)
    report['market_analysis'] = market_analysis.sort_values('Revenue', ascending=False).head(5)
    
    # Category Performance
    category_analysis = df.groupby('category').agg({
        'price': ['count', 'sum', 'mean'],
        'review_score': 'mean'
    }).round(2)
    category_analysis.columns = ['Orders', 'Revenue', 'AOV', 'Rating']
    category_analysis['Category_Share_%'] = (category_analysis['Revenue'] / category_analysis['Revenue'].sum() * 100).round(1)
    report['category_analysis'] = category_analysis.sort_values('Revenue', ascending=False).head(5)
    
    # Price Segment Analysis
    segment_analysis = df.groupby('price_segment').agg({
        'price': ['count', 'sum', 'mean'],
        'review_score': 'mean'
    }).round(2)
    segment_analysis.columns = ['Orders', 'Revenue', 'AOV', 'Rating']
    segment_analysis['Segment_Share_%'] = (segment_analysis['Revenue'] / segment_analysis['Revenue'].sum() * 100).round(1)
    report['segment_analysis'] = segment_analysis
    
    # Monthly Trends
    monthly_trends = df.groupby('month_name').agg({
        'price': ['count', 'sum'],
        'review_score': 'mean'
    }).round(2)
    monthly_trends.columns = ['Orders', 'Revenue', 'Rating']
    report['monthly_trends'] = monthly_trends
    
    return report

# Generate the report
bi_report = generate_business_report(df)

# Display the report
print("" + "="*60)
print("          COMPREHENSIVE BUSINESS INTELLIGENCE REPORT")
print("" + "="*60)

# Executive Summary
es = bi_report['executive_summary']
print(f"\n📊 EXECUTIVE SUMMARY")
print(f"   Total Orders: {es['total_orders']:,}")
print(f"   Total Revenue: R$ {es['total_revenue']:,.2f}")
print(f"   Average Order Value: R$ {es['avg_order_value']:.2f}")
print(f"   Customer Satisfaction: {es['customer_satisfaction']:.2f}/5.0 ({es['satisfaction_rate']:.1f}% satisfied)")

# Market Analysis
print(f"\n🗺️  TOP 5 MARKETS BY REVENUE")
for i, (state, row) in enumerate(bi_report['market_analysis'].iterrows(), 1):
    print(f"   {i}. {state}: R$ {row['Revenue']:,.2f} ({row['Market_Share_%']}%, AOV: R$ {row['AOV']:.2f})")

# Category Performance
print(f"\n🛍️  TOP 5 CATEGORIES BY REVENUE")
for i, (category, row) in enumerate(bi_report['category_analysis'].iterrows(), 1):
    print(f"   {i}. {category}: R$ {row['Revenue']:,.2f} ({row['Category_Share_%']}%, Rating: {row['Rating']:.1f})")

# Segment Analysis
print(f"\n💰 PRICE SEGMENT PERFORMANCE")
for segment, row in bi_report['segment_analysis'].iterrows():
    print(f"   {segment}: {row['Orders']:,} orders, R$ {row['Revenue']:,.2f} ({row['Segment_Share_%']}%)")

# Key Recommendations
print(f"\n🎯 KEY STRATEGIC RECOMMENDATIONS")
top_market = bi_report['market_analysis'].index[0]
top_category = bi_report['category_analysis'].index[0]
best_segment = bi_report['segment_analysis']['AOV'].idxmax()

print(f"   1. Invest in {top_market} market (top revenue generator)")
print(f"   2. Expand {top_category} category offerings (best performer)")
print(f"   3. Focus marketing on {best_segment} segment (highest AOV)")
print(f"   4. Address satisfaction in underperforming segments")

print("\n" + "="*60)

## Section 7: Key Takeaways and Best Practices

### 🎯 Advanced Concepts Mastered:

1. **Multi-Column GroupBy**: `df.groupby(['col1', 'col2']).agg()`
2. **Custom Aggregation Functions**: Create business-specific metrics
3. **Pivot Tables**: `pd.pivot_table()` for multi-dimensional analysis
4. **Complex Filtering**: Multiple conditions with boolean indexing
5. **Business Intelligence Reports**: Combining multiple techniques

### 🔄 Excel to Pandas Advanced Translation:

- **Excel Pivot Table** → `pd.pivot_table()`
- **Excel SUMIFS** → `df[multiple_conditions]['column'].sum()`
- **Excel Calculated Fields** → Custom aggregation functions
- **Excel Slicers** → Boolean indexing with multiple conditions

### 💡 Advanced Business Applications:

- Multi-dimensional market analysis
- Customer segmentation with complex criteria
- Executive dashboard creation
- Competitive performance benchmarking
- Strategic recommendation generation

### 🚀 Tomorrow's SQL Connection:
In tomorrow's SQL class, we'll perform these same multi-dimensional analyses using advanced GROUP BY, HAVING, and window functions!

## Advanced Practice Exercises

Challenge yourself with these advanced exercises:

1. **Multi-Level Analysis**: Create a pivot table showing satisfaction levels by state and category
2. **Custom KPI Function**: Write a function that calculates customer lifetime value indicators
3. **Market Opportunity Analysis**: Find states with high AOV but low order volumes
4. **Seasonal Patterns**: Analyze monthly performance by price segments
5. **Competitive Analysis**: Compare performance metrics across different business dimensions

In [ ]:
# Space for advanced practice exercises
# Students can implement complex business analyses here

print("Ready for advanced practice! Try the exercises above.")